# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
    The PEFT config i have chosen is `LORA` as i feel it is a safe bet and a good place to start. It is also very popular and widely trusted and used. it is also supported my chosen model.
* Model: 
    The model i have chosen is `distilbert/distilbert-base-uncased` with a `TokenClassification` head, based on the Popular `google-bert/bert-base-uncased` but smaller and faster than bert but with comparable accuracy.
* Evaluation approach: 
    The evaluation method i have chosen is `seqeval` which is used for for sequence labeling evaluation and will help me evaluate my chosen data set consisting of ner tags. Uses industry standards like the `GLUE` benchmark
* Fine-tuning dataset: 
    The `Dataset` i have chosen is `WNUT 17: Emerging and Rare entity recognition` a valid data set to use.


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

### Imports


In [34]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from datasets import load_dataset
import evaluate
import torch.nn.functional as F
import torch
from peft import LoraConfig, TaskType
from peft import get_peft_model
from peft import AutoPeftModelForSequenceClassification


### Load dataset

In [2]:

dataset = load_dataset("rotten_tomatoes")


Generating test split: 100%|██████████| 1066/1066 [00:00<00:00, 494648.53 examples/s]


### Inspect dataset

In [3]:
# Look at dataset
print("TRAIN:", dataset)
print("TEXT", dataset["train"].features[f"text"])
print("LABEL", dataset["train"].features[f"label"])

TRAIN: DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})
TEXT Value(dtype='string', id=None)
LABEL ClassLabel(names=['neg', 'pos'], id=None)


In [7]:
# Look at an example
example = dataset["train"][0]
print("EXAMPLE:",example)
print("TEXT:",example["text"])
print("LABEL:",example["label"])

EXAMPLE: {'text': 'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .', 'label': 1}
TEXT: the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
LABEL: 1


### Create label2id, id2label and label_count

In [8]:
# Create label2id, id2label and label_count
labels = dataset["train"].features[f"label"].names
id2label={}
label2id={}
for i, name in enumerate(labels):
    id2label[i] = name
    
for i, name in enumerate(labels):
    label2id[name] = i
    
label_count = len(labels);
print(id2label)
print(label2id)
print(label_count)

{0: 'neg', 1: 'pos'}
{'neg': 0, 'pos': 1}
2


### Load Tokenizer and Model

In [9]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=label_count, id2label=id2label, label2id=label2id
)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print(model.config)
print(model)
print(tokenizer)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertConfig {
  "_name_or_path": "distilbert/distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "neg",
    "1": "pos"
  },
  "initializer_range": 0.02,
  "label2id": {
    "neg": 0,
    "pos": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.32.1",
  "vocab_size": 30522
}

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (trans

### Inspect Tokenizer

In [10]:
example = dataset["train"][0]
tokenized_input = tokenizer(example["text"], truncation=True, max_length=200)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print("IN", example["text"])
print("OUT", tokens)
# word_ids is used to Map output tokens to their respective input word in the raw examples["tokens"]
print("WORD_IDS", tokenized_input.word_ids())
print("len(tokens)", len(tokens))
print("len(word_ids)", len(tokenized_input.word_ids()))

for i, (token, word_id) in enumerate(zip(tokens, tokenized_input.word_ids())):
    print(f"({token},{word_id})", end=", ")


IN the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
OUT ['[CLS]', 'the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'century', "'", 's', 'new', '"', 'conan', '"', 'and', 'that', 'he', "'", 's', 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarz', '##ene', '##gger', ',', 'jean', '-', 'cl', '##aud', 'van', 'dam', '##me', 'or', 'steven', 'sega', '##l', '.', '[SEP]']
WORD_IDS [None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 29, 29, 30, 31, 32, 33, 33, 34, 35, 35, 36, 37, 38, 38, 39, None]
len(tokens) 47
len(word_ids) 47
([CLS],None), (the,0), (rock,1), (is,2), (destined,3), (to,4), (be,5), (the,6), (21st,7), (century,8), (',9), (s,10), (new,11), (",12), (conan,13), (",14), (and,15), (that,16), (he,17), (',18), (s,19), (going,20), (to,21), (make,22), (a,2

### Align labels with tokens, taking into account the special tokens added by tokenizer

In [11]:
def tokenize_and_align_labels(examples):
    tokenized = tokenizer(examples["text"], padding="max_length", truncation=True)
    tokenized["labels"] = examples["label"]
    return  tokenized


tokenized_input = dataset.map(tokenize_and_align_labels, batched=True)
text = tokenized_input["train"][0]["text"]
inputIds = tokenized_input["train"][0]["input_ids"]
labels = tokenized_input["train"][0]["labels"]
print("TEXT:",text)
print("CONVERTED_TOKENS:",tokenizer.convert_ids_to_tokens(inputIds))
print("INPUT_IDS:",inputIds)
print("LABELS:",labels)

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Map: 100%|██████████| 1066/1066 [00:00<00:00, 39907.25 examples/s]


TEXT: the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .
CONVERTED_TOKENS: ['[CLS]', 'the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'century', "'", 's', 'new', '"', 'conan', '"', 'and', 'that', 'he', "'", 's', 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarz', '##ene', '##gger', ',', 'jean', '-', 'cl', '##aud', 'van', 'dam', '##me', 'or', 'steven', 'sega', '##l', '.', '[SEP]']
INPUT_IDS: [101, 1996, 2600, 2003, 16036, 2000, 2022, 1996, 7398, 2301, 1005, 1055, 2047, 1000, 16608, 1000, 1998, 2008, 2002, 1005, 1055, 2183, 2000, 2191, 1037, 17624, 2130, 3618, 2084, 7779, 29058, 8625, 13327, 1010, 3744, 1011, 18856, 19513, 3158, 5477, 4168, 2030, 7112, 16562, 2140, 1012, 102]
LABELS: 1


### Prepare Evalutation Logic


In [12]:
seqeval = evaluate.load("seqeval")

In [13]:
import numpy as np
label_list = dataset["train"].features[f"label"].names
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

### Evaluate model before training;
Evaluate a test example before training the model.
Grab an example with a low accuracy to test after training the model

Build trainer and then evaluate method

In [14]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_input["test"] = tokenized_input["test"].remove_columns(["text"])
training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_input["train"],
    eval_dataset=tokenized_input["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [15]:
trainer.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
100%|██████████| 67/67 [00:00<00:00, 158.78it/s]


{'eval_loss': 0.6927236914634705,
 'eval_accuracy': 0.5178236397748592,
 'eval_runtime': 0.5088,
 'eval_samples_per_second': 2094.988,
 'eval_steps_per_second': 131.674}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [16]:
#["lin1","lin2","classifier"]
peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, target_modules=["lin1"], inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

In [17]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters();

trainable params: 776,450 || all params: 67,731,460 || trainable%: 1.1463653669948943


### Train

In [18]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_input["train"],
#     eval_dataset=tokenized_input["test"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )
trainer.train()

 48%|████▊     | 511/1068 [00:07<00:07, 69.69it/s]

{'loss': 0.5359, 'learning_rate': 1.0636704119850187e-05, 'epoch': 0.94}


                                                  
 50%|█████     | 534/1068 [00:07<00:07, 68.68it/s]

{'eval_loss': 0.4237791895866394, 'eval_accuracy': 0.8133208255159474, 'eval_runtime': 0.4555, 'eval_samples_per_second': 2340.315, 'eval_steps_per_second': 147.093, 'epoch': 1.0}


 94%|█████████▍| 1009/1068 [00:15<00:00, 70.53it/s]

...

                                                   
100%|██████████| 1068/1068 [00:17<00:00, 68.80it/s]

{'eval_loss': 0.416022390127182, 'eval_accuracy': 0.8208255159474672, 'eval_runtime': 0.4624, 'eval_samples_per_second': 2305.154, 'eval_steps_per_second': 144.883, 'epoch': 2.0}


100%|██████████| 1068/1068 [00:18<00:00, 58.66it/s]

{'train_runtime': 18.2049, 'train_samples_per_second': 937.11, 'train_steps_per_second': 58.666, 'train_loss': 0.4736343215913808, 'epoch': 2.0}


TrainOutput(global_step=1068, training_loss=0.4736343215913808, metrics={'train_runtime': 18.2049, 'train_samples_per_second': 937.11, 'train_steps_per_second': 58.666, 'train_loss': 0.4736343215913808, 'epoch': 2.0})

In [19]:
trainer.save_model("rotten_model")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

Here i use the index for the test example with the lowest accuracy found before training. 

In [35]:
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("rotten_model", num_labels=label_count, id2label=id2label, label2id=label2id)

ValueError: Can't find 'adapter_config.json' at 'rotten_model'

In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_input["train"],
    eval_dataset=tokenized_input["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

100%|██████████| 67/67 [00:00<00:00, 151.16it/s]


{'eval_loss': 0.416022390127182,
 'eval_accuracy': 0.8208255159474672,
 'eval_runtime': 0.4595,
 'eval_samples_per_second': 2319.662,
 'eval_steps_per_second': 145.795}

In [31]:
positive_review = "I was deeply moved by this film. The storytelling was exceptional, and the characters felt like old friends. The chemistry between the lead actors was palpable, and the cinematography captured the beauty of the setting. The emotional depth and genuine moments left me smiling long after the credits rolled. A must-watch!"
negative_review = "I had high hopes for this movie, but it fell flat. The plot was clichéd, and the dialogue felt forced. The pacing was sluggish, and the acting lacked authenticity. I found myself checking my watch, waiting for something interesting to happen. Unfortunately, it never did. Save your time and skip this one."

input = tokenizer([positive_review,negative_review], truncation=True, padding=True, return_tensors="pt")
print(input)

input.to(device)
lora_model.to(device)
output = lora_model(**input)
print(output.logits.shape)
predictions = torch.argmax(
    output.logits, dim=0
)  

predicted_labels = [
    model.config.id2label[prediction.item()] for prediction in predictions
]
for review, label in zip([positive_review,negative_review], predicted_labels):
    print(f"LABEL: {label}, REVIEW: {review}")
# for i, z in zip(predicted_labels, tokenizer.tokenize(text)):
#     print(f"({z},{i})", end=", ")


{'input_ids': tensor([[  101,  1045,  2018,  2152,  8069,  2005,  2023,  3185,  1010,  2021,
          2009,  3062,  4257,  1012,  1996,  5436,  2001, 18856, 17322,  2094,
          1010,  1998,  1996,  7982,  2371,  3140,  1012,  1996, 15732,  2001,
         23667, 17701,  2232,  1010,  1998,  1996,  3772, 10858, 21452,  1012,
          1045,  2179,  2870,  9361,  2026,  3422,  1010,  3403,  2005,  2242,
          5875,  2000,  4148,  1012,  6854,  1010,  2009,  2196,  2106,  1012,
          3828,  2115,  2051,  1998, 13558,  2023,  2028,  1012,   102],
        [  101,  1045,  2001,  6171,  2333,  2011,  2023,  2143,  1012,  1996,
         20957,  2001, 11813,  1010,  1998,  1996,  3494,  2371,  2066,  2214,
          2814,  1012,  1996,  6370,  2090,  1996,  2599,  5889,  2001, 14412,
          4502,  3468,  1010,  1998,  1996, 16434,  4110,  1996,  5053,  1997,
          1996,  4292,  1012,  1996,  6832,  5995,  1998, 10218,  5312,  2187,
          2033,  5629,  2146,  2044,  1996, 

As you can see we went from an accuracy of 0.0 to 0.96, an 96% increase for the test sample, showing the model has learnt to predict the correct labels.

This is greater than expected and i have been over the code and i cant see anything wrong.

Im not sure why im getting such good results for the test example ( which the model hasn't seen and ) but not for the golden gate example.